In [1]:
import numpy as np
import pandas as pd
import re

import transliterate as tr

In [2]:
df = pd.read_csv('cars.csv', index_col='Unnamed: 0')

# Data Cleaning

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Marka   10000 non-null  object
 1   Model   10000 non-null  object
 2   Year    10000 non-null  object
 3   Price   10000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 390.6+ KB


In [4]:
df['Marka'].unique()

array(['Toyota', 'УАЗ', 'Kia', 'Zeekr', 'Land', 'Dodge', 'Audi', 'Mini',
       'Porsche', 'Jeep', 'Maserati', 'BMW', 'Mercedes-Benz', 'Hyundai',
       'Mitsubishi', 'Honda', 'ВАЗ', 'Nissan', 'Mazda', 'Subaru', 'Lexus',
       'Suzuki', 'Chevrolet', 'Volkswagen', 'Ford', 'Cadillac', 'BYD',
       'Opel', 'Infiniti', 'Volvo', 'Changan', 'Genesis', 'ГАЗ',
       'SsangYong', 'Lotus', 'Voyah', 'Hongqi', 'Daewoo', 'Tesla',
       'Jaguar', 'Skoda', 'EXEED', 'HiPhi', 'Avatr', 'IM', 'Li',
       'Bentley', 'ЗАЗ', 'Foton', 'Renault', 'JAC', 'Haval', 'Geely',
       'Peugeot', 'Hummer', 'Lincoln', 'Ravon', 'FAW', 'Mercedes-Maybach',
       'Lamborghini', 'Lifan', 'Chery', 'Smart', 'Jin', 'Daihatsu',
       'Citroen', 'Acura', 'Isuzu', 'Rolls-Royce', 'Neta', 'Wuling',
       'Fiat', 'GMC', 'Jetour', 'GAC', 'Seres', 'Tank', 'Leap', 'OMODA',
       'Ретро-автомобили', 'Denza', 'DongFeng', 'Москвич', 'Weltmeister',
       'BAIC', 'Aston', 'Chrysler', 'MG', 'Great', 'Gonow', 'Pontiac'],
      dtyp

In [5]:
df[df.Marka.str.contains('Land')]

,Marka,Model,Year,Price
13,Land,['Rover Range Rover'],2021,51000000
14,Land,['Rover Range Rover Sport'],2007,9000000
72,Land,['Rover Range Rover'],2006,11000000
82,Land,['Rover Discovery Sport'],2019,20500000
195,Land,['Rover Range Rover'],2014,24500000
...,...,...,...,...
9888,Land,['Rover Range Rover'],2005,6000000
9938,Land,['Rover Range Rover'],2016,34800000
9944,Land,['Rover Range Rover'],2016,34800000
9973,Land,['Rover Range Rover'],2007,9350000


In [6]:
df.Model = df.Model.apply(lambda x: x[2:-2])
df.loc[df.Marka == 'Land', 'Model'] = df.loc[df.Marka == 'Land', 'Model'].apply(lambda x: x[6:])
df.loc[df.Marka == 'Land', 'Marka'] = 'Land Rover'
print(df[df.Marka.str.contains('Land')].head())

          Marka              Model  Year     Price
13   Land Rover        Range Rover  2021  51000000
14   Land Rover  Range Rover Sport  2007   9000000
72   Land Rover        Range Rover  2006  11000000
82   Land Rover    Discovery Sport  2019  20500000
195  Land Rover        Range Rover  2014  24500000


In [7]:
df.Model.unique()

array(['Vellfire', 'Alphard', 'Estima', '3303', 'K5', '001',
       'Range Rover', 'Range Rover Sport', 'Challenger', 'Tacoma', 'A5',
       'Hatch', 'Cayenne', 'Grand Cherokee', 'Quattroporte', 'X5 M',
       'E 220', 'Elantra', 'Carina E', 'Highlander', 'Sonata',
       'Land Cruiser Prado', '100', 'E 320', 'Montero Sport', 'C 180',
       'Concerto', '(Lada) 2114 (хэтчбек)', 'Terrano', 'Patrol', 'X5',
       'X6', 'Cronos', 'Outback', 'CLS 55 AMG', 'Cefiro', 'LX 570',
       'S 500', 'Camry', 'SX4', 'Panamera', 'Teana', 'S 63 AMG',
       'Land Cruiser', '(Lada) Priora 2170 (седан)', 'Cruze',
       'Transporter', 'C 280', 'S 600', 'X', 'E 260', '525', 'Accent',
       'Avensis', 'L200', 'GLE 63 AMG', 'Mustang', 'Starex', 'GS 350',
       '540', 'Escalade', 'X-Trail', 'Discovery Sport', 'e:NS1', 'Han',
       'Santa Fe', 'Legacy', 'ML 350', 'Sorento', 'Corolla', 'RX 350',
       'M-NV', '3', 'X2', 'E 200', 'Vectra', '740', 'ID.4', 'Maxima',
       'Outlander', 'QX80', 'Tundra', 'GS 

In [8]:
df[df.Model.str.contains('\(')].Marka.unique()

array(['ВАЗ', 'ГАЗ', 'Ford', 'Renault'], dtype=object)

In [9]:
df.drop(df[df.Marka.str.contains('Ретро')].index, inplace=True)

In [11]:
df.loc[df.Model.str.contains(r'\('), 'Model'] = \
    df.loc[df.Model.str.contains(r'\('), 'Model'].replace(r"\s?\(.*?\)\s?", "", regex=True)

In [14]:
def transl(text):
    if re.search(r'[А-Яа-я]', text):
        return tr.translit(text, reversed=True)
    return text

In [15]:
df.Marka, df.Model = df.Marka.apply(transl), df.Model.apply(transl)

In [17]:
df[df.Marka.str.contains(r'[^A-Za-z-\s]')].Marka.unique()

array([], dtype=object)

In [19]:
df.Year.unique()

array(['2009', '2006', '2012', '2010', '1987', '2013', '2021', '2023',
       '2007', '2019', '2005', '2002', '2004', '2014', '1992', '2003',
       '1991', '2000', '2008', '1993', '2016', '2022', '2011', '1995',
       '1990', '1994', '1996', '2017', 'На з', '2001', '2020', '2018',
       '2015', '1997', '1999', '1998', '1976', '1986', '1989', '1984',
       '1988', '1983', '1977', '1973', '1981', '1985', '1982', '1972',
       '1954', '1963', '1959', '1956', '1979', '1957', '1970', '1969'],
      dtype=object)

In [34]:
df.drop(df[df.Year == 'На з'].index, inplace=True)

# Data Normalization and to_csv

In [40]:
brands = pd.DataFrame({'Brand Name': df['Marka'].unique()})
brands['Brand ID'] = range(len(brands))

models = pd.DataFrame({'Model Name': df['Model'].unique()})
models['Model ID'] = range(len(models))

car_full = df.copy()
car_full['Brand ID'] = car_full['Marka'].map(brands.set_index('Brand Name')['Brand ID'])
car_full['Model ID'] = car_full['Model'].map(models.set_index('Model Name')['Model ID'])

In [44]:
car_full

,Marka,Model,Year,Price,Brand ID,Model ID
0,Toyota,Vellfire,2009,7700000,0,0
1,Toyota,Alphard,2006,6500000,0,1
2,Toyota,Vellfire,2009,7700000,0,0
3,Toyota,Estima,2012,6300000,0,2
4,Toyota,Estima,2012,6500000,0,2
...,...,...,...,...,...,...
9995,Toyota,HiAce,1995,3800000,0,280
9996,Land Rover,Range Rover,2007,9350000,4,6
9997,BMW,X6 M,2021,100000000,11,170
9998,Mercedes-Benz,S 350,2006,6200000,12,130


In [45]:
brands.to_csv('brands.csv', index = False)
models.to_csv('models.csv', index = False)
car_full.to_csv('car_full.csv', index = False)

In [47]:
car_full[2520:2525]

,Marka,Model,Year,Price,Brand ID,Model ID
2594,Kia,K7,2021,15850000,2,148
2595,Nissan,Pathfinder,2012,10000000,17,346
2596,Toyota,Highlander,2021,28200000,0,19
2597,Toyota,Yaris,2010,5200000,0,477
2598,Zeekr,X,2023,17150000,3,74
